In [1]:
import re
from requests import Session
from lxml.html import fromstring
from urllib.parse import urljoin

In [2]:
BASE_URL = 'https://www.arasuper.com.br/'
ALL_DEPS = '//ul[@class="menu-principal__items menu-principal__col menu-principal__col--1"]'
NAME = 'a/span/span'
URL = 'a/@href'
SUBs = 'div/div/ul/li'

In [3]:
# INICIAR SESSION PARA TESTES
session = Session()

In [4]:
def get_tree_from_url (session, url):
    response = session.get(url)
    return fromstring(response.text)

# Departments

In [5]:
def get_departments(session):
    departments = {}
    tree = get_tree_from_url(session, BASE_URL)
    for dep in tree.xpath(ALL_DEPS)[1]:
        depName = dep.xpath(NAME)[0].text_content()    
        subDeps = dep.xpath(SUBs)
        
        if subDeps != []: # Exists subDeps
            for sub in subDeps:
                subcontent = sub.text_content().split()
                
                if not(subcontent in [['Todos'],['Voltar']]):     # Erro Dept Todos e Voltar
                    subName = sub.xpath(NAME)[0].text_content()
                    subSubDeps = sub.xpath(SUBs)
                    
                    if subSubDeps != []: # Exists subSubDeps
                        for subsub in subSubDeps:
                            subsubcontent = subsub.text_content().split()
                            
                            if not(subsubcontent in [['Todos'],['Voltar']]): # Insert DEP > SUB > SUBSUB
                                subSubName = subsub.xpath('a/div')[1].text_content().strip()
                                url = BASE_URL + subsub.xpath(URL)[0]
                                departments[url] = depName + ' > ' + subName + ' > ' + subSubName

                    else: # Insert DEP > SUB
                        url = BASE_URL + sub.xpath(URL)[0]
                        departments[url] = depName + ' > ' + subName

        else: # Insert DEP
            url = BASE_URL + dep.xpath(URL)[0]
            departments[url] = depName
    
    return departments
        

In [6]:
get_departments(session)

{'https://www.arasuper.com.br/c/massas-resfriadas/580/': 'Alimentos > Massas Resfriadas',
 'https://www.arasuper.com.br/c/torradas/562/': 'Alimentos > Torradas',
 'https://www.arasuper.com.br/c/diet/547/': 'Alimentos > Sal > Diet',
 'https://www.arasuper.com.br/c/queijo-ralado/530/': 'Alimentos > Queijo Ralado',
 'https://www.arasuper.com.br/c/soja/490/': 'Alimentos > Óleo > Soja',
 'https://www.arasuper.com.br/c/milho/489/': 'Alimentos > Óleo > Milho',
 'https://www.arasuper.com.br/c/girassol/488/': 'Alimentos > Óleo > Girassol',
 'https://www.arasuper.com.br/c/coco/487/': 'Alimentos > Óleo > Coco',
 'https://www.arasuper.com.br/c/composto/486/': 'Alimentos > Óleo > Composto',
 'https://www.arasuper.com.br/c/canola/485/': 'Alimentos > Óleo > Canola',
 'https://www.arasuper.com.br/c/mostarda/481/': 'Alimentos > Mostarda',
 'https://www.arasuper.com.br/c/polpa-de-tomate/475/': 'Alimentos > Polpa de Tomate',
 'https://www.arasuper.com.br/c/molho-tomate/474/': 'Alimentos > Molho Tomate',


# Products Information

In [9]:
def get_pages(url):
    i = 1
    while True:
        new_url = url + '?page=' + str(i)
        tree = get_tree_from_url(session, new_url)
        page_empty = tree.xpath('//div[@class="produto-lista empty-content"]')
        if page_empty != []:
            break
        else:
            yield tree
        i = i + 1

In [10]:
TEST_URL = 'https://www.arasuper.com.br/c/vinhos/351/'

for i in get_pages(TEST_URL):
    print(i)

<Element html at 0x17010133d10>
<Element html at 0x170101a1f40>
<Element html at 0x1700f108770>
<Element html at 0x1700f12f950>
<Element html at 0x17010133d10>
<Element html at 0x1700f14f0e0>


# Debug DEPT

In [10]:
TEST_URL = 'https://www.arasuper.com.br/c/vinhos/351/'

In [11]:
# New Get_pages
def get_pages(url):
    i = 1
    while True:
        new_url = url + '?page=' + str(i)
        tree = get_tree_from_url(session, new_url)
        page_empty = tree.xpath('//div[@class="produto-lista empty-content"]')
        if page_empty != []:
            break
        else:
            yield tree
        i = i + 1

In [12]:
for i in get_pages(TEST_URL):
    print(i)

<Element html at 0x1a163369b30>
<Element html at 0x1a163343090>
<Element html at 0x1a163343770>
<Element html at 0x1a163333d60>
<Element html at 0x1a163369090>


In [15]:
def get_page_products(page):
        return page.xpath(PRODUCT_XPATH)

In [16]:
def extract_price(price):
    return int(re.sub(r"\s|R|\$|,", "", price)) / 100

In [12]:
NAME_XPATH = 'div/div[3]/div/p[@class="item-produto__name"]'
BRAND_XPATH = 'div/div[3]/div/p[@class="item-produto__brand"]'
PRODUCT_ONE_XPATH = 'div/div[3]/div/p[@class="item-produto__price just-one"]'
PRODUCT_TWO_XPATH = 'div/div[3]/div/p[@class="item-produto__price"]'
PRODUCT_LEVE_XPATH = 'div/div[3]/div/p[@class="item-produto__price price-leve"]'
REGULAR_PRICE_XPATH = 'span[@class="item-produto__price-de"]'
PRICE_XPATH = 'span[@class="item-produto__price-por"]'
PRODUCT_XPATH =  '//div[@class="products-list"]/a'

def extract_product (product):
    product_url = BASE_URL + product.xpath('@href')[0]
    product_name = product.xpath(NAME_XPATH)[0].text_content()
    product_brand = product.xpath(BRAND_XPATH)[0].text_content()
    
    if product.xpath(PRODUCT_ONE_XPATH): # One price
        regular_price = None
        price = product.xpath(PRODUCT_ONE_XPATH)[0].text_content()
        price = extract_price(price)
        
    elif product.xpath(PRICES_TWO): # Regular Price and Price
        prices = product.xpath(PRICES_TWO)[0]
        regular_price = prices.xpath(REGULAR_PRICE_XPATH)[0].text_content()
        regular_price = extract_price(regular_price)
        price = prices.xpath(PRICE_XPATH)[0].text_content()
        price = extract_price(price)
    
    else: # Only price
        prices = product.xpath(PRICES_LEVE)[0]
        regular_price = prices.xpath(REGULAR_PRICE_XPATH)[0].text_content()
        regular_price = extract_price(regular_price)
        price = prices.xpath(PRICE_XPATH)[0].text_content()
        price = extract_price(price)
    
    
    element = {'url': product_url, 'name': product_name, 'brand': product_brand, 'price': price,'regular_price': regular_price}

    
    return {'url': product_url,
            'name': product_name,
            'brand': product_brand,
            'price': price,
            'regular_price': regular_price}

In [17]:
session = Session()
#test_url = 'https://www.arasuper.com.br/c/atum-e-sardinha/140/'
test_url = 'https://www.arasuper.com.br/c/destilados/179/'  # existe leve

for page in get_pages(test_url):
    print('PAGE : ',page)
    for product_element in get_page_products(page):
        print("PRODUCT : ", product_element)
        print(extract_product(product_element))

    print('\n')

PAGE :  <Element html at 0x1a162e818b0>
PRODUCT :  <Element a at 0x1a1632ad810>
{'url': 'https://www.arasuper.com.br/p/bebida-gin-rocks-sunset-1l/19551/', 'name': 'Bebida Gin Rocks Sunset 1L ', 'brand': 'ROCKS', 'price': 48.9, 'regular_price': None}
PRODUCT :  <Element a at 0x1a164351720>
{'url': 'https://www.arasuper.com.br/p/bebida-vodka-natural-absolut-750ml/19263/', 'name': 'Bebida Vodka Natural Absolut 750ML', 'brand': 'ABSOLUT', 'price': 79.98, 'regular_price': None}
PRODUCT :  <Element a at 0x1a164351180>
{'url': 'https://www.arasuper.com.br/p/bebida-aguardente-segredo-chacara-ouro-900ml/18585/', 'name': 'Bebida Aguardente Segredo Chacara Ouro 900ML. ', 'brand': 'SEGREDO DA CHACARA', 'price': 24.98, 'regular_price': None}
PRODUCT :  <Element a at 0x1a164351310>
{'url': 'https://www.arasuper.com.br/p/bebida-aguardente-segredo-chacara-900ml/18587/', 'name': 'Bebida Aguardente Segredo Chacara 900ML', 'brand': 'SEGREDO DA CHACARA', 'price': 24.98, 'regular_price': None}
PRODUCT :  <

PAGE :  <Element html at 0x1a164351d60>
PRODUCT :  <Element a at 0x1a163316630>
{'url': 'https://www.arasuper.com.br/p/gin-rocks-1l/6777/', 'name': 'Gin Rocks 1L', 'brand': 'ROCKS', 'price': 48.9, 'regular_price': None}
PRODUCT :  <Element a at 0x1a164351ef0>
{'url': 'https://www.arasuper.com.br/p/whisky-white-horse-garrafa-1l/6720/', 'name': 'Whisky White Horse Garrafa 1L', 'brand': 'WHITE HORSE', 'price': 112.9, 'regular_price': None}
PRODUCT :  <Element a at 0x1a164351f90>
{'url': 'https://www.arasuper.com.br/p/cachaca-cabare-ouro-garrafa-700ml/5704/', 'name': 'Cachaça Cabaré Ouro Garrafa 700Ml', 'brand': 'CABARE', 'price': 45.99, 'regular_price': None}
PRODUCT :  <Element a at 0x1a164351cc0>
{'url': 'https://www.arasuper.com.br/p/gin-gordons-750ml/5663/', 'name': 'Gin Gordons 750Ml', 'brand': 'GORDONS', 'price': 99.99, 'regular_price': None}
PRODUCT :  <Element a at 0x1a1643514a0>
{'url': 'https://www.arasuper.com.br/p/cachaca-velho-barreiro-910ml/4966/', 'name': 'Cachaça Velho Bar